幅広くデータを見ていく
* 加速度の波形を並べていみる
* ウィンドウフレームごとの平均，分散の変化

In [1]:
# 表示用
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

In [2]:
import numpy as np
from numpy import zeros, newaxis
import datetime
import os
import copy
import time

# Parametor about Sensor
SensorNum=1 # Sensor number
#SensorAxis='AccX' # Axis

# Parametor about  " Sliding Window " 
WindowWidth =np.array([128,256,512,1024,2048,4096]) # Window Width
SlidingWidth =WindowWidth/4 #sliding window

# Parametor about Neural Network
AEDimention = 16

# Define Data Name
DATE= str( datetime.date.today() )+'/'
TITLE="Participants01_First_1/"

# Parametor about " Data Path"
DataPath = "/media/takeyama/Transfer/02_ActivityResearch/Paticipants01/01_First/20161018-105301/mem/"

#chopin
#StorePath = "/home/takeyama/Documents/"
#mozart
StorePath ="/media/takeyama/HD-PNFU3/01_ActivityResearchData/"
if not os.path.exists(StorePath+TITLE+'dictionary/'): 
    os.makedirs(StorePath+TITLE+'dictionary/')
DictionaryDataPath=StorePath+TITLE+'dictionary/'

if not os.path.exists(StorePath+TITLE+'window/'): 
    os.makedirs(StorePath+TITLE+'window/')
WindowDataPath=StorePath+TITLE+'window/'

if not os.path.exists(StorePath+TITLE+DATE+'studyOutput/'): 
    os.makedirs(StorePath+TITLE+DATE+'studyOutput/')
StudyOutputPath=StorePath+TITLE+DATE+'studyOutput/'

if not os.path.exists(StorePath+TITLE+DATE+'parametor/'): 
    os.makedirs(StorePath+TITLE+DATE+'parametor/')
ParametorPath=StorePath+TITLE+DATE+'parametor/'

if not os.path.exists(StorePath+TITLE+DATE+'result/'): 
    os.makedirs(StorePath+TITLE+DATE+'result/')
ResultPath=StorePath+TITLE+DATE+'result/'

if not os.path.exists(StorePath+'/'+TITLE+DATE+'graph/'): 
    os.makedirs(StorePath+'/'+TITLE+DATE+'graph/')
GraphPath=StorePath+'/'+TITLE+DATE+'graph/'

if not os.path.exists(StorePath+'/'+TITLE+DATE+'graph/corrMap/'): 
    os.makedirs(StorePath+'/'+TITLE+DATE+'graph/corrMap/')
CorrMapGraphPath=StorePath+'/'+TITLE+DATE+'graph/corrMap/'

if not os.path.exists(StorePath+'/'+TITLE+'feature/'): 
    os.makedirs(StorePath+'/'+TITLE+'feature/')
FeaturePath=StorePath+'/'+TITLE+'feature/'

OSError: [Errno 13] Permission denied: '/media/takeyama/HD-PNFU3/01_ActivityResearchData'

センサデータの読み込み

In [ ]:
# センサデータのcsvファイル名をリストに集約する関数
def SensorDataFileNameList(path):
    l = os.listdir(DataPath)
    FilePath =[]
    SensorName = []
    
    for n in range(len(l)):
        if (l[n][:4].find('mem-') != -1):
            FilePath.append(l[n])
            SensorName.append(l[n][4:15])
    
    return FilePath,SensorName

CSVを読み込んで，CSVの中身を辞書型で表現する

In [ ]:
# FFT library
from scipy import fftpack
from scipy import signal

# Graph Plot
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns

# Generate WindowFrame Function
from mymodule import window

# timer
import time

# temp file 
import tempfile

# pandas
import pandas as pd

%matplotlib inline

In [ ]:
DictName = os.listdir(DictionaryDataPath)
print DictName

In [ ]:
Sensor5 = np.load(DictionaryDataPath+DictName[0])['data'][()]
Sensor1 = np.load(DictionaryDataPath+DictName[1])['data'][()]
Sensor3 = np.load(DictionaryDataPath+DictName[2])['data'][()]
Sensor2 = np.load(DictionaryDataPath+DictName[3])['data'][()]
Sensor4 = np.load(DictionaryDataPath+DictName[4])['data'][()]
Sensor6 = np.load(DictionaryDataPath+DictName[5])['data'][()]

In [ ]:
def GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH):
    """
    data　ウィンドウフレームに変換するデータ
    registName　ウィンドウフレームに登録するデータの名前
    windowWidth　ウィンドウ幅
    slidingWidth　スライド幅
    PATH ウィドウフレームを保存するディレクトリ
    
    
    About Function:
        与えられたdataからウィンドウ幅windowWidth,スライド幅slidingWidthにしたがって
        registNameのウィドウフレームを返す．
        また，与えれたPATH内に同様なパラメータ( WindowWidth, slidingWidth)かつ同様な
        windowNameのものがある場合，そのデータを返す．
        この関数が登録，ウィンドウフレームに変換できるデータは１つとする
    """
    storedName = windowName+'_Win='+str(wWidth).zfill(4)+'_Sld='+str(sWidth).zfill(4)+'.npz'
    l = os.listdir(PATH)
    
    if storedName in l:
        print "this data had finished making"
        return np.load(PATH+storedName)['data'][()]
    
    w=window()
    w.SetData(windowName,dataRaw)    
    wind=w.Compile(wWidth,sWidth)
    windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))
    
    np.savez(PATH+storedName,data=wind)

    return wind

# Methematical/Statistical Functions

## Mean, Median

In [ ]:
def FastVector_Mean(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].mean() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Mean==========='
    return Output

In [ ]:
def FastVector_Median(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [np.median(wind[i,:]) for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Median==========='
    return Output

## Variance, Std Deviation

In [ ]:
def FastVector_Var(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].var() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Variance==========='
    return Output

In [ ]:
def FastVector_StdDev(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)  
    Output = np.array( [wind[i,:].var()**0.5 for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of standard Deviation==========='
    return Output

## Min, Max, Range
Range = the difference between maximum and minimum sample values

In [ ]:
def FastVector_Max(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].max() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Max==========='
    return Output

In [ ]:
def FastVector_Min(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].min() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Min==========='
    return Output

In [ ]:
def FastVector_Range(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [(wind[i,:].max()-wind[i,:].min())\
                        for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Range==========='
    return Output

## RMS

In [ ]:
def FastVector_RMS(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [((np.sum( wind[i,:]**2)/wind[i,:].size)**0.5)\
                        for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Root Means Square==========='
    return Output

## Correlation, Cross-Correlation

In [ ]:
def FastVector_Corr(windowNameX, dataRawX, windowNameY, dataRawY, wWidth, sWidth, PATH):
    # get windowFrame
    windX=GetWindowFrame(windowNameX, dataRawX, wWidth, sWidth, PATH)
    windY=GetWindowFrame(windowNameY, dataRawY, wWidth, sWidth, PATH)
    Output = np.array( [( np.corrcoef(windX[i,:],windY[i,:]) )\
                    for i in range(windX.shape[0]) ] )
    print '===========finished Making Vector of Correlation==========='
    return Output[:,0,1]

# Signal magnitude area

In [ ]:
def FastVector_SMA(windowName, dataRawX, dataRawY, dataRawZ, wWidth, sWidth, PATH):
    # get windowFrame
    windX=GetWindowFrame(windowName, dataRawX, wWidth, sWidth, PATH)
    windY=GetWindowFrame(windowName, dataRawY, wWidth, sWidth, PATH)
    windZ=GetWindowFrame(windowName, dataRawZ, wWidth, sWidth, PATH)
    Output = np.array( [(np.sum((windX[i,:]**2+windY[i,:]**2+windZ[i,:]**2)**0.5))\
                for i in range(windX.shape[0]) ] )
    print '===========finished Making Vector of Signal Magnitude Area==========='
    return Output  

# 各センサの相関を求める

In [ ]:
labeledData=np.load(FeaturePath+'Label_Sensor1.npz')['data'][()]

In [ ]:
w=window()
w.SetData('DataAddingLabel', labeledData[7,:])    
wind=w.Compile(256, 32)
windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))

In [ ]:
# 重複するリストをまとめて，リストにする
def remove_duplicates(x):
    y=[]
    for i in x:
        if i not in y:
            y.append(i)
    return y

In [ ]:
remove_duplicates(windoW[0])

In [ ]:
for s in remove_duplicates(windoW[0]):
    print s

In [ ]:
len( np.where(windoW[0]==u'\u505c\u6b62')[0] )

In [ ]:
def RemoveDuplicates(seq):
    seen = set()
    seen_add = seen.add
    return [ x for x in seq if x not in seen and not seen_add(x)]

In [ ]:
RemoveDuplicates(windoW[0])

In [ ]:
[np.unique(windoW[i,:]) for i in range(windoW.shape[0]) if np.unique(windoW[i,:]).size > 1]

In [ ]:
[i for i in range(windoW.shape[0]) if np.unique(windoW[i,:]).size > 1]

In [ ]:
windoW[516]

In [ ]:
np.unique(windoW[516]).size

In [ ]:
[np.unique(windoW[516])[0]for i in range(np.unique(windoW[516]).size)]

In [ ]:
# 複数のラベルがあるウィンドウ番号で，要素数が多いラベルがnp.uniqueで生成する配列のどの番号にあるか示す
np.argmax(np.array([len( np.where(windoW[516]==np.unique(windoW[516])[i])[0])for i in range(np.unique(windoW[516]).size)]))

In [ ]:
len( np.where(windoW[516]==np.unique(windoW[516][1]))[0])

In [ ]:
for winNum in range(7,16):
    print "***********winNum="+str(winNum)+"***********"
    w=window()
    w.SetData('DataAddingLabel', labeledData[winNum,:])    
    wind=w.Compile(256, 32)
    windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))

    LabelArray=np.array([np.unique(windoW[l,:])[
            np.argmax(np.array([len( np.where(windoW[l]==np.unique(windoW[l])[i])[0])for i in range(np.unique(windoW[l]).size)]))]
             for l in range(windoW.shape[0])
    ])

    print LabelArray

In [ ]:
def MakeWindowFrameLabel(LabelData1d, wWidth, sWidth):
    w=window()
    w.SetData('DataAddingLabel', LabelData1d)    
    wind=w.Compile(wWidth, sWidth)
    windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))

    LabelArray=np.array([np.unique(windoW[l,:])[
            np.argmax(np.array([len( np.where(windoW[l]==np.unique(windoW[l])[i])[0])for i in range(np.unique(windoW[l]).size)]))]
             for l in range(windoW.shape[0])
    ])
    return LabelArray

In [ ]:
test= MakeWindowFrameLabel(labeledData[15,:], 256, 32)
print test.shape

In [ ]:
df = pd.DataFrame({
    'LeftHand-Verb':MakeWindowFrameLabel(labeledData[7,:], 256, 32),
    'LeftHand-Object':MakeWindowFrameLabel(labeledData[8,:], 256, 32),
    'RightHand-Verb':MakeWindowFrameLabel(labeledData[9,:], 256, 32),
    'RightHand-Object ':MakeWindowFrameLabel(labeledData[10,:], 256, 32),
    'Leg-Number':MakeWindowFrameLabel(labeledData[11,:], 256, 32),
    'Leg-Verb':MakeWindowFrameLabel(labeledData[12,:], 256, 32),
    'Leg-Object':MakeWindowFrameLabel(labeledData[13,:], 256, 32),
    'West':MakeWindowFrameLabel(labeledData[14,:], 256, 32),
    'ActiveNumber':MakeWindowFrameLabel(labeledData[15,:], 256, 32),
    })

In [ ]:
df

In [ ]:
drop_df = df.drop('ActiveNumber',axis=1).drop_duplicates()

In [ ]:
drop_df

In [ ]:
for c in df.columns:
    print 'columns='+c+' unique='+str( len( df[c].unique() ) )

In [ ]:
for key in df['ActiveNumber'].unique():
    print 'Columns='+str(key)+' count='+str(df[df['ActiveNumber']==key].shape[0] )

In [ ]:
ActinName=np.array(['行動開始',
'時計の運搬',
'雑誌の整理',
'雑誌，本棚の掃除',
'雑誌，本棚の掃除で用いた道具の片付け',
'雑巾がけ',
'雑巾の片付け',
'時計の片付け',
'ホウキを用いた掃除',
'ホウキの片付け',
'掃除機を用いた掃除',
'掃除位の片付け',
'モップシートの交換',
'モップを用いた掃除',
'モップの片付け'])


In [ ]:
"""
ウィンドウ幅 256
スライド幅 32
上記の設定でウィドウフレームを生成した

そして，各行動別にどのくらいのラベルがあるのかを調べた結果が以下の
通りである．
"""
for AN,key in zip(ActinName, df['ActiveNumber'].unique() ):
    print 'ActionName='+AN+'__'+str(df[df['ActiveNumber']==key].shape[0] )
    
KindOfLabel = np.array([df[df['ActiveNumber']==key].shape[0] for key in df['ActiveNumber'].unique()])

In [ ]:
KindOfLabel

In [ ]:
plt.bar(xrange(KindOfLabel[1:].shape[0]), KindOfLabel[1:], tick_label=ActinName)